Nesta fase, irei iniciar a etapa de experimentação, onde irei testar uma série 
de modelos de classificação para encontrar o que melhor se adequa ao problema.
Vale ressaltar que apenas algumas colunas serão utilizadas para a criação dos
modelos, pois algumas só são obtidas ao fim do processo do SISU, como é o caso
da coluna "NOTA_CORTE" e "CLASSIFICACAO". Outras colunas como códigos das IES
e cursos também não serão utilizadas, pois não são relevantes para o problema.
Ao final, além do modelo, um conjunto de dados no formato .db será gerado para
ser consumido pelo app final. Segue abaixo as colunas que serão utilizadas:

- Modelo: IES, UF_CAMPUS, MUNICIPIO_CAMPUS, NOME_CURSO, GRAU, TURNO, 
TIPO_MOD_CONCORRENCIA, QT_VAGAS_CONCORRENCIA, PERCENTUAL_BONUS, PESO_L, PESO_CH,
PESO_CN, PESO_M, PESO_R, NOTA_MINIMA_L, NOTA_MINIMA_CH, NOTA_MINIMA_CN, 
NOTA_MINIMA_M, NOTA_MINIMA_R, MEDIA_MINIMA, OPCAO, NOTA_L, NOTA_CH, NOTA_CN, 
NOTA_M, NOTA_R, NOTA_L_COM_PESO, NOTA_CH_COM_PESO, NOTA_CN_COM_PESO, 
NOTA_M_COM_PESO, NOTA_R_COM_PESO, NOTA_CANDIDATO e APROVADO.

Vale ressaltar que parte das informações que serão utilizadas no Web App serão 
buscadas nos dados do SISU, como é o caso da QT_VAGAS_CONCORRENCIA, que é um 
valor que a universidade define para cada curso e não o usuário. Outras serão 
calculadas manualmente, como no caso das notas com peso.


In [1]:
import mlflow
import pandas as pd
import category_encoders as ce

# Preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, PowerTransformer, OneHotEncoder, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import log_loss


In [2]:
# Lendo os dados
dados_sisu = pd.read_parquet('../data/processed/dados_transformados.parquet/')

In [3]:
# Definindo as colunas que serão utilizadas para treinar o modelo
colunas_para_buscar = ['IES', 'UF_CAMPUS', 'MUNICIPIO_CAMPUS', 'NOME_CURSO', 
                       'GRAU', 'TURNO', 'TIPO_MOD_CONCORRENCIA', 
                       'QT_VAGAS_CONCORRENCIA', 'PERCENTUAL_BONUS', 'PESO_L', 
                       'PESO_CH', 'PESO_CN', 'PESO_M', 'PESO_R', 
                       'NOTA_MINIMA_L', 'NOTA_MINIMA_CH', 'NOTA_MINIMA_CN', 
                       'NOTA_MINIMA_M', 'NOTA_MINIMA_R', 'MEDIA_MINIMA', 
                       'OPCAO', 'NOTA_L', 'NOTA_CH', 'NOTA_CN', 'NOTA_M', 
                       'NOTA_R', 'NOTA_L_COM_PESO', 'NOTA_CH_COM_PESO', 
                       'NOTA_CN_COM_PESO', 'NOTA_M_COM_PESO', 'NOTA_R_COM_PESO',
                       'NOTA_CANDIDATO', 'APROVADO']

In [4]:
# Filtrando as colunas
dados_sisu = dados_sisu[colunas_para_buscar]

In [5]:
# Verificando o balanceamento da variável alvo
dados_sisu['APROVADO'].value_counts(normalize=True)

APROVADO
N    0.898245
S    0.101755
Name: proportion, dtype: float64

In [6]:
# Define o local para salvar os exoerimentos
mlflow.set_tracking_uri('../mlruns')

# Criando/acessando o experimento
mlflow.set_experiment('Comparando modelos')

2023/11/30 17:10:49 INFO mlflow.tracking.fluent: Experiment with name 'Comparando modelos' does not exist. Creating a new experiment.


<Experiment: artifact_location='/home/daniel/Documents/sisu_analysis/notebooks/../mlruns/121856067394284381', creation_time=1701375049985, experiment_id='121856067394284381', last_update_time=1701375049985, lifecycle_stage='active', name='Comparando modelos', tags={}>

In [7]:
# Dividindo os dados em variaveis explicativas e variavel alvo
x = dados_sisu.drop(columns=['APROVADO'])
y = dados_sisu['APROVADO'].map({'S': 1, 'N': 0})
y

# Dividindo os dados em treino e teste
x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size=0.2, random_state=42, stratify=y)

# Dividindo os dados em teste e dev
x_teste, x_dev, y_teste, y_dev = train_test_split(x_teste, y_teste, test_size=0.5, random_state=42, stratify=y_teste)

# Dividindo os dados em dev e calibração
x_dev, x_calibracao, y_dev, y_calibracao = train_test_split(x_dev, y_dev, test_size=0.5, random_state=42, stratify=y_dev)

In [8]:
# Dividindo as colunas por tipo
high_dim_cols = ['MUNICIPIO_CAMPUS', 'IES', 'NOME_CURSO']
num_cols = x_treino.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = x_treino.select_dtypes(include=['object']).columns.tolist()
cat_cols = [col for col in cat_cols if col not in high_dim_cols]

# Criando o kfolds
kfolds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [9]:


# Criando um scaler padrão
scale = y_treino.value_counts()[0] / y_treino.value_counts()[1] 

# Criando dicionário com os modelos
dict_models_scale_sensitive = {"LR": LogisticRegression(random_state=200,
                                                        class_weight='balanced')}

dict_models_tree_based = {"LGBM": LGBMClassifier(is_unbalance=True,
                                                 random_state=200),
                          "XGB": XGBClassifier(scale_pos_weight=scale,
                                               random_state=200),
                          "CTBC": CatBoostClassifier(auto_class_weights='Balanced',
                                                     random_state=200),
                          "DT": DecisionTreeClassifier(class_weight='balanced',
                                                       random_state=200),
                          "RF": RandomForestClassifier(class_weight='balanced',
                                                       random_state=200)}

# Criando dicionário com os encoders
dict_encoders = {"OHE": OneHotEncoder(drop='first'),
                 "TE": ce.TargetEncoder(),
                 "ME": ce.MEstimateEncoder(),
                 "WOE": ce.WOEEncoder(),
                 "CE": ce.CatBoostEncoder(),
                 "GE":ce.GrayEncoder()}

dict_scalers = {"SS": StandardScaler()}

# Criando dicionário com os transformers
dict_transformers = {"PT": PowerTransformer(),
                     "PF": PolynomialFeatures()}

In [10]:
# Iniciando os experimentos sem transformers
for tag, model in dict_models_scale_sensitive.items():
    for tag_encoder, encoder in dict_encoders.items():
        for tag_scaler, scaler in dict_scalers.items():
            
            # Gerando a tag de identificação do modelo
            nome_modelo = f'{tag}_{tag_encoder}_{tag_scaler}'
            
            with mlflow.start_run(run_name=nome_modelo):
                 
                 # Criando os pipeline com os transformers
                 pipe_cat = Pipeline([('encoder', encoder)])
                 pipe_high_dim = Pipeline([('encoder', ce.CountEncoder())])
                 pipe_num = Pipeline([('scaler', scaler)])
                 
                 # Criando o transformador
                 transformer = ColumnTransformer([('cat', pipe_cat, cat_cols),
                                                 ('num', pipe_num, num_cols),
                                                 ('high_dim', pipe_high_dim, high_dim_cols)])
                 
                 # Criando o pipeline final
                 pipe = Pipeline([('transformer', transformer),
                                 ('model', model)])
                 
                 # Executando o cross validation
                 cross_val_scores = cross_val_score(pipe, x_treino, y_treino, cv=kfolds, scoring='neg_log_loss')
                 
                 # Calculando a média das métricas
                 mean_score = cross_val_scores.mean()           
                 
                 # Salvando a métrica da folder 1
                 mlflow.log_metric('log_loss_fold_1', cross_val_scores[0])
                 
                 # Salvando a métrica da folder 2
                 mlflow.log_metric('log_loss_fold_2', cross_val_scores[1])
                
                 # Salvando a métrica da folder 3
                 mlflow.log_metric('log_loss_fold_3', cross_val_scores[2])
                
                 # Salvando a métrica da folder 4
                 mlflow.log_metric('log_loss_fold_4', cross_val_scores[3])
                
                 # Salvando a métrica da folder 5
                 mlflow.log_metric('log_loss_fold_5', cross_val_scores[4])
                 
                 # Salvando as métricas
                 mlflow.log_metric('log_loss_mean', mean_score)


: 